In [1]:
import sys
sys.path.append('../../lib/')

In [2]:
from analysis import *
import json
import requests
import time
import shutil

In [3]:
with open('../../key.json') as f:
	key = json.load(f)
	bearer_token = key['BEARER_TOKEN']

In [5]:
tweets_list_possable_bad = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/posable_bad_tweets.json"))
tweets_list_good = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/300_random_users_first_tweet.json"))
tweets_list_false_positives = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/false_postives_from_v3_ivory.json"))
seed_user_good_tweets = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/100_random_tweets_from_seed_users.json"))

tweets_list = tweets_list_possable_bad + tweets_list_good + tweets_list_false_positives + seed_user_good_tweets

In [35]:
def get_tweet(tweet_id):
	# Create the url
	url = 'https://api.twitter.com/2/tweets/{}'.format(tweet_id) + '?expansions=attachments.media_keys,referenced_tweets.id.author_id&&media.fields=url&&tweet.fields=id&&user.fields=description'
	# Create request headers
	headers = {
		'authorization': 'Bearer ' + bearer_token,
		'user-agent': 'v2FilteredStreamPython',
	}
	# Make the request
	response = requests.get(url, headers=headers)
	return response.json()

In [36]:
tweets_with_user_ids = {}
tweets_with_user_ids["tweets"] = []
# Go through each tweet
for tweet in tweets_list:
    try:
        tweet_response = get_tweet(analysis.get_tweet_id(tweet))
        if tweet_response['data']:
            tweets_with_user_ids["tweets"].append(tweet_response)
    except:
        print(analysis.get_tweet_text(tweet)) 
    time.sleep(2)
    break

In [32]:
with open('../../data/dataset/tweets_with_user_ids.json', 'w') as f:
	json.dump(tweets_with_user_ids, f)